In [10]:
import pandas as pd
import gzip
import json
import numpy as np
import regex as re
import matplotlib.pyplot as plt
import seaborn as sns
import dask 
import dask.dataframe as dd
import imblearn
from collections import Counter

import re
import os
import sys
import math
import time
import nltk
import torch
import random
import string
import unidecode
import collections

import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from gensim.models import Word2Vec
from nltk.corpus import brown
from sklearn.manifold import TSNE
from torch.autograd import Variable
from torchtext import data, datasets
from torchtext.vocab import Vectors

import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

from IPython.display import Image, YouTubeVideo
from torch.nn import functional as F
from d2l import torch as d2l

from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import time
%matplotlib inline

ModuleNotFoundError: No module named 'ray'

In [8]:
pip install madgrad

Note: you may need to restart the kernel to use updated packages.


In [9]:
from madgrad import MADGRAD

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def HyperTrain(device = device, train_path = None, test_path =None, #file/device params 
               loader = None, preloaded = False, batch_size = 32, max_vocab= None, embed_vectors = "glove.6B.100d", fix_length = None, #loader params
               output_size = 2, hidden_layers = 100, embed_size = 100, dropout = 0, #model params
               epochs = 10, learning_rate = .001, weight_decay = 0, criterion = nn.CrossEntropyLoss(), optimizer = 'Adam',#training params
               choose_best_epoch = True, #testing params
               initial_weights = False, #weight initialization
               ):
    '''
  
    loader: use to switch to a different loader (if BERT needs a different loader style)
    preloaded: default False, set this to True if you already loaded your data and don't want to test any loader configurations
    dropout: proportion that should be dropped in dropout layers (0 is equivalent to no dropout)
    choose_best_epoch: default True. Set to True if you want the final test evaluation to be done on the model state saved from the epoch with the lowest validation loss
    initial_weights: choose if and how you want to initialize weights, can be bool or string of choices
    
    '''
    def load_dataset(batch_size = 32, max_vocab = None, embed_vectors = "glove.6B.100d", fix_length = None):
        TEXT = data.Field(tokenize = nltk.word_tokenize,
                          include_lengths = True, batch_first = True, fix_length = fix_length)
        LABEL = data.LabelField(dtype = torch.float)

        fields = {'reviewText': ('text', TEXT), 'pos_neg': ('label', LABEL)}
        train_data, test_data = data.TabularDataset.splits(
            path = '',
            train = train_path,
            test = test_path,
            format = 'json',
            fields = fields
        )

        
        #glove example: glove.6B.100d means we use 6 billion 100-dimensional glove embeddings 
        #use unk_init to set the unembedded words via Gaussian distribution
        TEXT.build_vocab(train_data, max_size = max_vocab, vectors = embed_vectors, unk_init = torch.Tensor.normal_)
        LABEL.build_vocab(train_data)

        train_data, valid_data = train_data.split(split_ratio=0.8, random_state = random.seed(42))

        #Use BucketIterator to split batches into reviews of similar length and pad each batch accordingly.
        #This will greatly speed up our processing by making us have to process way fewer non-useful pad tokens.
        train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data),
                                                                       batch_size= batch_size, sort_key=lambda x: len(x.text),
                                                                       repeat=False, shuffle=True, sort_within_batch = True)
        vocab_size = len(TEXT.vocab)
           
        #get number of samples for printing          
        train_samples = len(train_data)
        valid_samples = len(valid_data)
        test_samples =len(test_data)
        num_samples = train_samples + valid_samples + test_samples

        return TEXT, vocab_size, train_iter, valid_iter, test_iter, num_samples
    
    
    ##### PUT ARCHITECTURE HERE
    
    
    
    #####
    
    
    
    
    
    
    def train(model, device, train_iter, valid_iter, epochs, learning_rate, criterion = nn.CrossEntropyLoss(), optim = 'Adam'):
        if optim == 'Adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay = weight_decay)
        elif optim == 'Madgrad':
            optimizer =  MADGRAD(model.parameters(), lr=learning_rate, weight_decay = weight_decay)

        #We'll set up a best validation loss (set to infinity at first) so we can save the best epoch
        best_validation_loss = float('inf')
        train_loss, validation_loss = [], []
        train_acc, validation_acc = [], []

        for epoch in range(epochs):
            #train
            model.train()
            running_loss = 0.
            correct, total = 0, 0 
            steps = 0

            for idx, batch in enumerate(train_iter):
                text = batch.text[0]
              
                target = batch.label
                target = torch.autograd.Variable(target).long()
                text, target = text.to(device), target.to(device)

               
                optimizer.zero_grad()
                output = model(text)

                loss = criterion(output, target)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(bert_model.parameters(), HYPER_PARAMS["max_grad_norm"])
                optimizer.step()
                steps += 1
                running_loss += loss.item()

                # get accuracy 
                _, predicted = torch.max(output, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

            train_loss.append(running_loss/len(train_iter))
            train_acc.append(correct/total)

            print(f'Epoch: {epoch + 1},  Training Loss: {running_loss/len(train_iter):.4f}, Training Accuracy: {100*correct/total: .2f}%')

            # evaluate on validation data
            model.eval()
            running_loss = 0.
            correct, total = 0, 0 

            with torch.no_grad():
                for idx, batch in enumerate(valid_iter):
                    text = batch.text[0]
                    target = batch.label
                    target = torch.autograd.Variable(target).long()
                    text, target = text.to(device), target.to(device)

                    optimizer.zero_grad()
                    output = model(text)

                    loss = criterion(output, target)
                    running_loss += loss.item()

                    # get accuracy 
                    _, predicted = torch.max(output, 1)
                    total += target.size(0)
                    correct += (predicted == target).sum().item()

            v_loss = running_loss/len(valid_iter)
            validation_loss.append(v_loss)
            validation_acc.append(correct/total)

            print (f'Validation Loss: {v_loss:.4f}, Validation Accuracy: {100*correct/total: .2f}%')
            
            # If the current epoch has the lowest validation loss, save the model state and use that state for testing
            if v_loss < best_validation_loss:
                best_validation_loss = v_loss
                torch.save(model.state_dict(), 'RNN-train.pt')


        return train_loss, train_acc, validation_loss, validation_acc
    
    
    def test(model,  device, test_iter, choose_best_epoch = True):
        
        #Use epoch with lowest validation loss
        if choose_best_epoch:
            model.load_state_dict(torch.load('RNN-train.pt'))
            
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for idx, batch in enumerate(test_iter):
                text = batch.text[0]
                target = batch.label
                target = torch.autograd.Variable(target).long()
                text, target = text.to(device), target.to(device)

                outputs = model(text)
                _, predicted = torch.max(outputs, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        acc = 100 * correct / total
        return acc
    
    def plot_train_val(x, train, val, train_label, val_label, title):
        plt.plot(x, train, label=train_label)
        plt.plot(x, val, label=val_label)
        plt.legend(loc='lower right')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title(title)
        plt.show()

    def count_parameters(model):
        parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
        return parameters
    
    # set initial weights
    def init_weights(m):
        if type(m) in (nn.Linear, nn.Conv1d):
            nn.init.xavier_uniform_(m.weight)
            
    
    # Start the training based on hyperparams
    # Initialize model, training and testing
    model = architecture(output_size, hidden_size, vocab_size, embedding_length, dropout)
    rnn_model.to(device)
    start_time = time.time()
    train_loss, train_acc, validation_loss, validation_acc = train(rnn_model, device, train_iter, valid_iter, epochs, learning_rate)
    print("--- Training Time = %s seconds ---" % (time.time() - start_time))
    test_accuracy = test(rnn_model, device, test_iter)
    print('Test Accuracy: ',  test_accuracy, '%')

    # Plot accuracy curves
    plot_train_val(np.arange(0,epochs), train_acc, validation_acc,
                   'training accuracy', 'validation accuracy', f'{Model_name} on Binary (Positive/Negative), {num_samples} Samples')

    # Number of model parameters
    paramters = count_parameters(rnn_model)
    print('\n\nNumber of parameters = ',  paramters)

In [ ]:
#define hyperparameters:


    #Loading
embed_vectors = ["glove.6B.100d"]
fix_length = None


    # Training 
learning_rates = [.0005, .001, .005]
weight_decay = [0]
criterion = [nn.CrossEntropyLoss()]
optimizers = ['Adam', 'Madgrad']


    # Model
output_size = 2
hidden_layers = [200] 
embed_size = [100]
epochs = 10
dropout = [.3]

for e in embed_vectors:
    for lr in learning_rates:
        for wd in weight_decay:
            for c in criterion:
                for opt in optimizers:
                     for h in hidden_layers:
                         for es in embed_size:
                             for d in dropout:
                                 HyperTrain(embed_vectors = e, fix_length = fix_length, 
                                            learning_rate = lr, weight_decay = wd, criterion = c, optimizer = opt
                                            output_size = output_size, hidden_layers = h, embed_size = es, epochs = epochs, dropout = d)